# 模型构造

让我们回顾一下在[“多层感知机的简洁实现”](../chapter_deep-learning-basics/mlp-gluon.ipynb)一节中含单隐藏层的多层感知机的实现方法。我们首先构造`Sequential`实例，然后依次添加两个全连接层。其中第一层的输出大小为256，即隐藏层单元个数是256；第二层的输出大小为10，即输出层单元个数是10。我们在上一章的其他
节中也使用了`Sequential`类构造模型。这里我们介绍另外一种基于`tf.keras.Model`类的模型构造方法：它让模型构造更加灵活。


## 4.1.1 build model from block

`tf.keras.Model`类是`tf.keras`模块里提供的一个模型构造类，我们可以继承它来定义我们想要的模型。下面继承`tf.keras.Model`类构造本节开头提到的多层感知机。这里定义的`MLP`类重载了`tf.keras.Model`类的`__init__`函数和`call`函数。它们分别用于创建模型参数和定义前向计算。前向计算也即正向传播。

In [1]:
import tensorflow as tf
import numpy as np
print(tf.__version__)

2.18.0


TensorFlow自定义的多层感知机（MLP）类

1. **类的定义**：
```python
class MLP(tf.keras.Model):
```
- 继承自tf.keras.Model基类
- 这允许我们自定义神经网络的结构和行为

2. **初始化方法**：
```python
def __init__(self):
    super().__init__()
    self.flatten = tf.keras.layers.Flatten()
    self.dense1 = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
    self.dense2 = tf.keras.layers.Dense(units=10)
```

- 调用父类的初始化方法
- 定义三个层：
  - Flatten层：将输入展平（例如将图像从2D转为1D）
  - 第一个全连接层：256个神经元，使用ReLU激活函数
  - 第二个全连接层：10个神经元（可能用于10分类问题）

3. **前向传播方法**：
```python
def call(self, inputs):         
    x = self.flatten(inputs)   
    x = self.dense1(x)    
    output = self.dense2(x)     
    return output
```

- 定义数据在网络中的流动路径
- 数据依次经过：
  1. 展平层：将多维输入转为一维
  2. 第一个全连接层：进行非线性变换
  3. 第二个全连接层：生成最终输出


In [2]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)

    def call(self, inputs):
        x = self.flatten(inputs)
        x = self.dense1(x)
        output = self.dense2(x)
        return output

以上的`MLP`类中无须定义反向传播函数。系统将通过自动求梯度而自动生成反向传播所需的`backward`函数。

我们可以实例化`MLP`类得到模型变量`net`。下面的代码初始化`net`并传入输入数据`X`做一次前向计算。其中，`net(X)`将调用`MLP`类定义的`call`函数来完成前向计算。

In [3]:
X = tf.random.uniform((2,20))
net = MLP()
result = net(X)
print(result.shape)

(2, 10)


## 4.1.2 Sequential

我们刚刚提到，`tf.keras.Model`类是一个通用的部件。事实上，`Sequential`类继承自`tf.keras.Model`类。当模型的前向计算为简单串联各个层的计算时，可以通过更加简单的方式定义模型。这正是`Sequential`类的目的：它提供`add`函数来逐一添加串联的`Block`子类实例，而模型的前向计算就是将这些实例按添加的顺序逐一计算。

我们用Sequential类来实现前面描述的MLP类，并使用随机初始化的模型做一次前向计算。

In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
])

model(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.05120257, -0.19586492,  0.37992322, -0.08915935, -0.13434957,
        -0.3283069 ,  0.16965775, -0.1765847 , -0.46149057,  0.00329264],
       [ 0.14645079, -0.11623782,  0.3985425 , -0.00321507,  0.00680106,
        -0.45539084,  0.0901586 , -0.20427983, -0.23858397,  0.04572239]],
      dtype=float32)>

## 4.1.3 build complex model

虽然`Sequential`类可以使模型构造更加简单，且不需要定义`call`函数，但直接继承`tf.keras.Model`类可以极大地拓展模型构造的灵活性。下面我们构造一个稍微复杂点的网络`FancyMLP`。在这个网络中，我们通过`constant`函数创建训练中不被迭代的参数，即常数参数。在前向计算中，除了使用创建的常数参数外，我们还使用`tensor`的函数和Python的控制流，并多次调用相同的层。















这是一个较为特殊的自定义神经网络类，包含了一些非常规的操作。让我详细解释其结构和特点：

1. **类的初始化**：
```python
def __init__(self):
    super().__init__()
    self.flatten = tf.keras.layers.Flatten()
    self.rand_weight = tf.constant(tf.random.uniform((20,20)))
    self.dense = tf.keras.layers.Dense(units=20, activation=tf.nn.relu)
```

- 定义了展平层
- 创建一个20×20的随机权重矩阵（固定不训练）
- 定义一个20个神经元的全连接层

2. **前向传播过程**：
```python
def call(self, inputs):         
    x = self.flatten(inputs)   # 展平输入
    # 自定义线性变换并激活
    x = tf.nn.relu(tf.matmul(x, self.rand_weight) + 1)
    x = self.dense(x)    # 全连接层处理
```

3. **特殊的输出处理**：
```python
    while tf.norm(x) > 1:
        x /= 2    # 如果范数大于1，不断除以2
    if tf.norm(x) < 0.8:
        x *= 10   # 如果范数小于0.8，乘以10
    return tf.reduce_sum(x)  # 返回所有元素的和
```

这个网络的特殊之处：

1. **固定权重**：
   - 使用了不可训练的随机权重矩阵
   - 这部分参数在训练过程中保持不变

2. **自定义激活**：
   - 手动实现了矩阵乘法和ReLU激活
   - 添加了常数偏置项1

3. **动态调整输出**：
   - 基于输出的范数动态调整数值
   - 实现了一种特殊的归一化机制

4. **标量输出**：
   - 最终将多维输出压缩为单个数值

这种设计的特点：
- 非常规的网络结构
- 包含了条件判断和循环
- 混合了固定和可训练的参数
- 实现了复杂的输出处理逻辑

这个网络是为了演示Keras的灵活性而设计的，实际应用中较少使用这样复杂的自定义逻辑。

In [5]:
class FancyMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.rand_weight = tf.constant(
            tf.random.uniform((20,20)))
        self.dense = tf.keras.layers.Dense(units=20, activation=tf.nn.relu)
    @tf.function
    def call(self, inputs):         
        x = self.flatten(inputs)   
        x = tf.nn.relu(tf.matmul(x, self.rand_weight) + 1)
        x = self.dense(x)    
        while tf.norm(x) > 1:
            x /= 2
        if tf.norm(x) < 0.8:
            x *= 10
        return tf.reduce_sum(x)

在这个`FancyMLP`模型中，我们使用了常数权重`rand_weight`（注意它不是模型参数）、做了矩阵乘法操作（`tf.matmul`）并重复使用了相同的`Dense`层。下面我们来测试该模型的随机初始化和前向计算。

In [6]:
net = FancyMLP()
net(X)
net.summary()


Model: "fancy_mlp"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_2 (Flatten)             │ (2, 20)                │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (2, 20)                │           420 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 420 (1.64 KB)

 Trainable params: 420 (1.64 KB)

 Non-trainable params: 0 (0.00 B)

因为`FancyMLP`和`Sequential`类都是`tf.keras.Model`类的子类，所以我们可以嵌套调用它们。















这是一个嵌套结构的多层感知机（MLP）类，让我详细解释其结构和特点：

1. **类的定义**：
```python
class NestMLP(tf.keras.Model):
```

- 继承自tf.keras.Model
- 实现了一个嵌套的神经网络结构

2. **初始化方法**：
```python
def __init__(self):
    super().__init__()
    self.net = tf.keras.Sequential()
    self.net.add(tf.keras.layers.Flatten())
    self.net.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
    self.net.add(tf.keras.layers.Dense(32, activation=tf.nn.relu))
    self.dense = tf.keras.layers.Dense(units=16, activation=tf.nn.relu)
```


- 创建了一个Sequential模型作为内部网络
- 内部网络包含：
  - Flatten层：将输入展平
  - 第一个全连接层：64个神经元，ReLU激活
  - 第二个全连接层：32个神经元，ReLU激活
- 额外定义了一个独立的全连接层：16个神经元，ReLU激活

3. **前向传播方法**：
```python
def call(self, inputs):
    return self.dense(self.net(inputs))
```


- 数据首先通过内部Sequential网络处理
- 然后通过额外的全连接层进行最终处理

这个网络的特点：
1. **嵌套结构**：
   - 在一个模型中嵌套了另一个模型
   - 展示了Keras模型的组合能力

2. **渐进式降维**：
   - 从输入到输出，特征维度逐渐减小
   - 64 -> 32 -> 16的结构

3. **统一的激活函数**：
   - 所有全连接层都使用ReLU激活函数
   - 有助于引入非线性特征

这种设计适用于：
- 需要层次化特征提取的场景
- 复杂的特征转换任务
- 模型结构需要模块化的场景

In [9]:
class NestMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.net = tf.keras.Sequential()
        self.net.add(tf.keras.layers.Flatten())
        self.net.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
        self.net.add(tf.keras.layers.Dense(32, activation=tf.nn.relu))
        self.dense = tf.keras.layers.Dense(units=16, activation=tf.nn.relu)

    
    def call(self, inputs):
        return self.dense(self.net(inputs))


In [10]:
net = tf.keras.Sequential()
net.add(NestMLP())
net.add(tf.keras.layers.Dense(20))
net.add(FancyMLP())
net(X)

<tf.Tensor: shape=(), dtype=float32, numpy=19.53986358642578>